In [6]:
from src.database import IngredientsDatabase
from src.inventory import Inventory
from src.player import Player
from src.ingredient import Ingredient
from src.alchemy_simulator import AlchemySimulator

import numpy as np
import time

In [7]:
db = IngredientsDatabase()

inv = Inventory.generate_random_weighted(db, size=10)

for ingredient in inv.get_available_ingredients():
    print(f"{ingredient} x {inv.get_quantity(ingredient)}")

Small Antlers x 8
Garlic x 8
Netch Jelly x 1
Scaly Pholiota x 20
Slaughterfish Egg x 2
Aloe Vera Leaves x 4
Spawn Ash x 3
Bleeding Crown x 5
Briar Heart x 3
Skeever Tail x 5


In [25]:
alembic = AlchemySimulator.from_base_player()
db = IngredientsDatabase()
results = []  # Initialize results list
start_time = time.time()
for i in range(20):  # Reduced from 1M for testing
    # size = np.random.randint(3, 50)
    size = 20
    inv = Inventory.generate_normal(db, size=size)
    alembic.set_inventory(inv)
    print(f"iter {i} --- {(time.time() - start_time)} seconds --- " )
    results.append(alembic.exhaust_inventory(strategy="greedy-basic"))
    alembic.delete_inventory()

print(f"Completed {len(results)} simulations")
print(f"Average potions per run: {sum(len(r) for r in results) / len(results):.2f}")

iter 0 --- 0.042777299880981445 seconds --- 
iter 1 --- 0.3232760429382324 seconds --- 
iter 2 --- 0.6705322265625 seconds --- 
iter 3 --- 0.9385263919830322 seconds --- 
iter 4 --- 1.2303333282470703 seconds --- 
iter 5 --- 1.5927343368530273 seconds --- 
iter 6 --- 1.9833171367645264 seconds --- 
iter 7 --- 2.2575533390045166 seconds --- 
iter 8 --- 2.443692207336426 seconds --- 
iter 9 --- 2.713839292526245 seconds --- 
iter 10 --- 3.209569215774536 seconds --- 
iter 11 --- 3.597475290298462 seconds --- 
iter 12 --- 3.916405200958252 seconds --- 
iter 13 --- 4.22001838684082 seconds --- 
iter 14 --- 4.800199270248413 seconds --- 
iter 15 --- 5.316064119338989 seconds --- 
iter 16 --- 5.975863218307495 seconds --- 
iter 17 --- 6.501615285873413 seconds --- 
iter 18 --- 7.034653186798096 seconds --- 
iter 19 --- 7.296926975250244 seconds --- 
Completed 20 simulations
Average potions per run: 42.60


In [27]:
for result in results:
    print("\tinventory results:\n\t==================\n")
    for potion in result:
        
        potion.print_self()
        

	inventory results:



Poison of Damage Magicka Regen
Ingredients: Boar Tusk, Burnt Spriggan Wood, Blue Mountain Flower
Total Value: 420 gold
Number of Effects: 3

Effects:
  - Health is increased by 17 points for 60 seconds.
  - Target moves at 50% speed for 22 seconds.
  - Decrease the target's Magicka regeneration by 100% for 22 seconds.


Poison of Damage Magicka Regen
Ingredients: Ancestor Moth Wing, Chaurus Hunter Antennae, Crimson Nirnroot
Total Value: 351 gold
Number of Effects: 4

Effects:
  - Does 39 points of Stamina damage.
  - Conjuration spells last 22% longer for 60 seconds.
  - Decrease the target's Magicka regeneration by 100% for 22 seconds.
  - Items are enchanted 4% stronger for 30 seconds.


Poison of Slow
Ingredients: Deathbell, Salt Pile, Fire Salts
Total Value: 302 gold
Number of Effects: 2

Effects:
  - Target moves at 50% speed for 22 seconds.
  - Magicka regenerates 22% faster for 300 seconds.


Poison of Slow
Ingredients: Deathbell, Salt Pile, Fire Salts
Tot